In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/email-spam-dataset/lingSpam.csv
/kaggle/input/email-spam-dataset/enronSpamSubset.csv
/kaggle/input/email-spam-dataset/completeSpamAssassin.csv


# Important Libraries

In [29]:
import numpy as np
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Making Functions to get the results 

In [30]:
# Load dataset
def load_dataset(file_path):
    """Load dataset from file."""
    return pd.read_csv(file_path)

# Clean 'Body' column
def clean_text(text):
    """Clean text by removing '\r\n' characters."""
    return text.replace('\r\n', ' ')

# Preprocess text data
def preprocess_text(text_data):
    """Preprocess text data."""
    corpus = []
    stemmer = PorterStemmer()
    stopwords_set = set(stopwords.words('english'))
    for text in text_data:
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation)).split()
        text = [stemmer.stem(word) for word in text if word not in stopwords_set]
        text = ' '.join(text)
        corpus.append(text)
    return corpus

# Vectorize text data
def vectorize_text(corpus):
    """Vectorize text data using CountVectorizer."""
    vectorizer = CountVectorizer()
    return vectorizer.fit_transform(corpus).toarray(), vectorizer

# Split data into training and test sets
def split_data(X, y, test_size=0.2, random_state=42):
    """Split data into training and test sets."""
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

# Train Naive Bayes model
def train_model(X_train, y_train):
    """Train Naive Bayes model."""
    model = MultinomialNB()
    model.fit(X_train, y_train)
    return model

# Evaluate the model
def evaluate_model(model, X_test, y_test):
    """Evaluate the model."""
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return accuracy, report

# Function to classify a single email
def classify_email(email_text, vectorizer, model):
    """Classify a single email."""
    stemmer = PorterStemmer()
    stopwords_set = set(stopwords.words('english'))
    email_text = email_text.lower().translate(str.maketrans('', '', string.punctuation)).split()
    email_text = [stemmer.stem(word) for word in email_text if word not in stopwords_set]
    email_text = ' '.join(email_text)
    email_corpus = [email_text]
    X_email = vectorizer.transform(email_corpus)
    prediction_label = model.predict(X_email)
    return prediction_label

# Step 2: processing code

In [31]:
# Load dataset
df = load_dataset('/kaggle/input/email-spam-dataset/lingSpam.csv')

# Clean 'Body' column
df['Body'] = df['Body'].apply(clean_text)

# Preprocess text data
corpus = preprocess_text(df['Body'])

# Vectorize text data
x, vectorizer = vectorize_text(corpus)
y = df['Label']

# Split data into training and test sets
X_train, X_test, y_train, y_test = split_data(x, y)

# Train Naive Bayes model
model = train_model(X_train, y_train)

# Evaluate the model
accuracy, report = evaluate_model(model, X_test, y_test)
print(f'Accuracy : {accuracy}')
print(f'Classification Report :\n{report}')

# Classify an email
email_to_classify = df['Body'].values[2600]
prediction_label = classify_email(email_to_classify, vectorizer, model)
print(f'Predicted Label: {prediction_label[0]}')
print(f'Actual Label: {df["Label"].iloc[2600]}')

Accuracy : 0.9942418426103646
Classification Report :
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       436
           1       1.00      0.96      0.98        85

    accuracy                           0.99       521
   macro avg       1.00      0.98      0.99       521
weighted avg       0.99      0.99      0.99       521

Predicted Label: 0
Actual Label: 0


# Save the model for later use 

In [32]:
import joblib

In [33]:
# Save the trained model
joblib.dump(model, 'naive_bayes_spam_model.pkl')

# Save the vectorizer
joblib.dump(vectorizer, 'count_vectorizer.pkl')

print("Model and Vectorizer saved successfully!")


Model and Vectorizer saved successfully!


In [34]:
# Load the trained model
loaded_model = joblib.load('naive_bayes_spam_model.pkl')

# Load the vectorizer
loaded_vectorizer = joblib.load('count_vectorizer.pkl')

# Now you can use the loaded model and vectorizer to classify new emails


In [35]:
# Classify a spam email
spam_email = "Congratulations! You’ve been selected for a chance to win a $1,000 Walmart gift card. Click the link below to claim your prize now!"
prediction_label_spam = classify_email(spam_email, vectorizer, model)
print(f'Predicted Label for Spam Email: {prediction_label_spam[0]}')

# Classify a ham (non-spam) email
ham_email = "Hi Team, This is a reminder for our team sync scheduled at 2 PM today. Please make sure to join the Zoom link sent earlier."
prediction_label_ham = classify_email(ham_email, vectorizer, model)
print(f'Predicted Label for Ham Email: {prediction_label_ham[0]}')


Predicted Label for Spam Email: 1
Predicted Label for Ham Email: 1
